## import modules


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl

## inputs


In [9]:
# i don't want to use other people's work so i'm going to leave power rating out of this model
# as well as all of the komputer and initial seed data
prev_seed = pd.read_csv("seed/538 Ratings.csv")
prev_seed = prev_seed.drop(['POWER RATING', 'POWER RATING RANK'], axis=1)
games = pd.read_csv("seed/games.csv")
how_teams_shoot = pd.read_csv("teams/Shooting Splits.csv")
coach = pd.read_csv("teams/Coach Results.csv")

## the model


In [53]:
# clustering
# what do the final four have in common
# can't cluster data that sits in an organized line like: 1 2 4 4 8 8 8 8 16 16 16 16 16 16 16 16 etc.

In [54]:
# association rules
# let it figure out where the connections are
# needs non numeric data

In [55]:
# # how strong is the wisdom of crowds


# # inputs
# crowd_wisdom_AP = pd.read_csv("votes/Preseason Votes.csv")
# crowd_wisdom_this_year = pd.read_csv("votes/Public Picks.csv")


# # imports
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

# # preprocessing
# crowd_wisdom_AP['YEAR'] -= 1
# merged = pd.merge(crowd_wisdom_AP, prev_seed, on=['YEAR', 'TEAM'],
#                   suffixes=('_crowd_wis', '_prev_seed'))
# merged = merged.drop(['TEAM NO_crowd_wis', 'TEAM NO_prev_seed',
#                      'ROUND_crowd_wis',
#                       'SEED_crowd_wis', 'SEED_prev_seed'], axis=1)

# # normalization
# scaler = MinMaxScaler()
# merged[['AP VOTES', 'AP RANK']] = scaler.fit_transform(
#     merged[['AP VOTES', 'AP RANK']])

# # split into features and target
# features = merged[['AP VOTES', 'AP RANK', 'RANK?']]
# target = merged['ROUND_prev_seed']

# # Split the data into training and testing sets
# test_size = 0.2 if len(features) > 5 else 0.3
# X_train, X_test, y_train, y_test = train_test_split(
#     features, target, test_size=test_size, random_state=42)

# # run the model
# model = LinearRegression()
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)

# # grade the results
# mse = mean_squared_error(y_test, predictions)
# print(mse)


# # answer is not very. wisdom of crowds works better when you take an average not when you vote with first past the post

In [50]:
# better decision tree

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import random


year = 2023
games_this_year = games[games['Year'] == year]
droppable_fields = ['YEAR', 'TEAM NO', 'TEAM ID', 'CONF']
droppable_fields.append('TEAM')

all_game_results = pd.DataFrame()
for i, element in enumerate(games_this_year.itertuples()):

    if random.choice([True, False]):
        this_game = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
            how_teams_shoot['TEAM'] == element.TeamLeft)].drop(droppable_fields, axis=1).add_suffix("_team1")

        other_team = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
            how_teams_shoot['TEAM'] == element.TeamRight)].drop(droppable_fields, axis=1).add_suffix("_team2")

        games_this_year.loc[i, 'TeamLeft'] = 1

    else:
        this_game = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
            how_teams_shoot['TEAM'] == element.TeamRight)].drop(droppable_fields, axis=1).add_suffix("_team1")

        other_team = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
            how_teams_shoot['TEAM'] == element.TeamLeft)].drop(droppable_fields, axis=1).add_suffix("_team2")

        games_this_year.loc[i, 'TeamLeft'] = 0

    this_game['target'] = element.TeamLeft

    for col in other_team.columns:
        this_game[col] = other_team[col].iloc[0]
    all_game_results = pd.concat([all_game_results, this_game])


# with pd.option_context('display.max_columns', None):
#     display(all_game_results)


data = all_game_results.drop('target', axis=1)
target = all_game_results['target']

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)
c45 = DecisionTreeClassifier()
c45.fit(X_train, y_train)
y_pred = c45.predict(X_test)

# evaluate accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# ?
# ?
# ?

foo = {
    'Year': [2023],
    'TeamLeft': ['Connecticut'],
    'TeamRight': ['Southeast Missouri St.'],
}

bar = pd.DataFrame()
for element in pd.DataFrame(foo).itertuples():

    this_game = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
        how_teams_shoot['TEAM'] == element.TeamLeft)].drop(droppable_fields, axis=1).add_suffix("_team1")

    other_team = how_teams_shoot[(how_teams_shoot['YEAR'] == year) & (
        how_teams_shoot['TEAM'] == element.TeamRight)].drop(droppable_fields, axis=1).add_suffix("_team2")

    for col in other_team.columns:
        this_game[col] = other_team[col].iloc[0]
    bar = pd.concat([bar, this_game])


print(f"test: {c45.predict(bar)}")

Accuracy: 0.23076923076923078
test: ['Connecticut']
